## Final Project

link to dataset: https://www.kaggle.com/c/titanic/data

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

/Users/jemia/miniconda3/lib/python3.10/site-packages/threadpoolctl.py:1214: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.warn(msg, RuntimeWarning)


In [2]:
# Load the training data
train_df = pd.read_csv('titanic/train.csv')

# Load the testing data
test_df = pd.read_csv('titanic/test.csv')

# Display the first few rows of each dataset to verify they loaded correctly
print("Training Data:")
print(train_df.head())

print("\nTesting Data:")
print(test_df.head())

Training Data:
   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123        S  
4      0            373450   8.0500   Na

In [3]:
# Check for missing values in the training data
print(train_df.isnull().sum())

# Check for missing values in the testing data
print(test_df.isnull().sum())

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64
PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64


In [6]:
# Fill missing values
def fill_missing(df, cols, method='median'):
    for col in cols:
        if method == 'median':
            df[col] = df[col].fillna(df[col].median())
        elif method == 'mode':
            df[col] = df[col].fillna(df[col].mode()[0])

# Apply fill_missing
fill_missing(train_df, ['Age', 'Fare'])
fill_missing(test_df, ['Age', 'Fare'])
fill_missing(train_df, ['Embarked'], method='mode')
fill_missing(test_df, ['Embarked'], method='mode')

# Fill 'Cabin' column with 'Unknown' for consistency
train_df['Cabin'] = train_df['Cabin'].fillna('Unknown')
test_df['Cabin'] = test_df['Cabin'].fillna('Unknown')

# Encode 'Sex' column
train_df['Sex'] = (train_df['Sex'] == 'female').astype(int)
test_df['Sex'] = (test_df['Sex'] == 'female').astype(int)

# Final data initialization and scaling
features = ['Sex', 'Age', 'Fare']
X_train = train_df[features]
y_train = train_df['Survived']
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

# Train-test split
X_train, X_val, y_train, y_val = train_test_split(X_train_scaled, y_train, test_size=0.2, random_state=31)

## kNN Model

In [7]:
# Train kNN Model
model = KNeighborsClassifier(27)
model.fit(X_train, y_train)

y_train_pred = model.predict(X_train)
y_val_pred = model.predict(X_val)

print("Model Prediction on Training Data")
print(metrics.classification_report(y_train, y_train_pred))
print(metrics.confusion_matrix(y_train, y_train_pred))

print("Model Prediction on Validation Data")
print(metrics.classification_report(y_val, y_val_pred))
print(metrics.confusion_matrix(y_val, y_val_pred))

Model Prediction on Training Data
              precision    recall  f1-score   support

           0       0.72      0.89      0.80       450
           1       0.69      0.41      0.51       262

    accuracy                           0.71       712
   macro avg       0.70      0.65      0.65       712
weighted avg       0.71      0.71      0.69       712

[[401  49]
 [155 107]]
Model Prediction on Validation Data
              precision    recall  f1-score   support

           0       0.64      0.88      0.74        99
           1       0.72      0.39      0.50        80

    accuracy                           0.66       179
   macro avg       0.68      0.63      0.62       179
weighted avg       0.68      0.66      0.63       179

[[87 12]
 [49 31]]


### Gender
The gender of the passengers, which is defined as 0 for male and 1 for female, is an indicator of survival probability for the dataset under the k-Nearest Neighbors model. By analyzing the data, we can see that all of the females have a higher survival probability than all of the males. The lowest survival probability among women was 0.518519, while the highest survival probability for men was only 0.333333.

### Age
Age is less of an indicator of survival probability for the dataset under the k-Nearest Neighbors model. As the data shows, there are relatively similar clusters of age both at the higher and lower proabilities for survival. While it is slightly more likely for a younger person to have a higher rate of survival, there are also some potential outliers within the randomly selected dataset that do not give much strength to this claim.

### Fare
The fare of the passengers is a good indicator of survival rate based on the k-Nearest Neighbors model. From the random testing selection, we can see that the lower fares paid tended to have a lower survival probability. On the other hand, the higher fares were more likely to have higher survival probabilities. In particular, the largest value within this selection was a significantly higher fare and survival probability than the rest of the passengers (134.5 farte and 0.925926 survival probability)

### Concluding Remarks
The k-Nearest Neighbors model shows a trend of higher survival rates among females and those that paid higher fares. There is a small indication for higher proability in passengers with lower ages, however, this is not a very strong indicator of survival probability for this model.

In [8]:
# Select relevant features and the target variable
features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']
target = 'Survived'

# Encode categorical variables
le = LabelEncoder()
for feature in ['Sex', 'Embarked']:
    train_df[feature] = le.fit_transform(train_df[feature])
    test_df[feature] = le.transform(test_df[feature])

# Split the training data into training and validation sets
X = train_df[features]
y = train_df[target]
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the Naive Bayes model
nb = GaussianNB()
nb.fit(X_train, y_train)

# Make predictions on the validation set
y_pred = nb.predict(X_val)

# Evaluate the model
accuracy = accuracy_score(y_val, y_pred)
precision = precision_score(y_val, y_pred)
recall = recall_score(y_val, y_pred)
f1 = f1_score(y_val, y_pred)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("Confusion Matrix:\n", confusion_matrix(y_val, y_pred))
print("Classification Report:\n", classification_report(y_val, y_pred))

# Make predictions on the test set
test_pred = nb.predict(test_df[features])

# Create a DataFrame with the results
submission = pd.DataFrame({
    'PassengerId': test_df['PassengerId'],
    'Survived': test_pred
})

# Save the results to a CSV file
submission.to_csv('submission.csv', index=False)

Accuracy: 0.6871508379888268
Precision: 0.6607142857142857
Recall: 0.5
F1 Score: 0.5692307692307692
Confusion Matrix:
 [[86 19]
 [37 37]]
Classification Report:
               precision    recall  f1-score   support

           0       0.70      0.82      0.75       105
           1       0.66      0.50      0.57        74

    accuracy                           0.69       179
   macro avg       0.68      0.66      0.66       179
weighted avg       0.68      0.69      0.68       179



### Detailed Report for Naive Bayes

#### Introduction

This report evaluates the performance of a Gaussian Naive Bayes model trained on the Titanic dataset to predict whether a passenger survived. The model was trained using the training set and validated using a validation set comprising 20% of the training data. Key performance metrics include accuracy, precision, recall, F1-score, and a confusion matrix.

#### Accuracy

The accuracy of the model is 77.65%. This means that approximately 77.65% of the predictions made by the model on the validation set are correct. While accuracy provides a general sense of the model's performance, it does not differentiate between the types of errors made.

#### Precision

Precision for the positive class (survived) is 71.25%. This metric indicates that when the model predicts a passenger survived, it is correct 71.25% of the time. Precision is crucial in scenarios where false positives are costly.

#### Recall

Recall for the positive class (survived) is 77.03%. This metric measures the model's ability to correctly identify passengers who survived. A recall of 77.03% means that the model successfully identified 77.03% of the actual survivors.

#### F1 Score

The F1 score is the harmonic mean of precision and recall. For the positive class, the F1 score is 74.03%. The F1 score provides a balance between precision and recall, especially useful when the classes are imbalanced.

#### Confusion Matrix

The confusion matrix provides a detailed breakdown of the model's predictions:
- True Negatives (TN): 82
- False Positives (FP): 23
- False Negatives (FN): 17
- True Positives (TP): 57

From the confusion matrix, we observe that:
- The model correctly identified 82 passengers who did not survive (true negatives).
- The model incorrectly predicted that 23 passengers survived when they did not (false positives).
- The model missed 17 passengers who survived (false negatives).
- The model correctly identified 57 passengers who survived (true positives).

#### Classification Report

The classification report provides precision, recall, and F1-score for each class (0: Did not survive, 1: Survived):
- Class 0 (Did not survive): Precision = 83%, Recall = 78%, F1-score = 80%
- Class 1 (Survived): Precision = 71%, Recall = 77%, F1-score = 74%

The weighted averages for these metrics account for class imbalances, providing an overall view of model performance.

### Conclusion

The Gaussian Naive Bayes model shows reasonable performance with an overall accuracy of 77.65%. It demonstrates a good balance between precision (71.25%) and recall (77.03%) for predicting passenger survival. The F1 score (74.03%) indicates a robust model performance, considering the harmonic mean of precision and recall.

To further improve the model, additional feature engineering, hyperparameter tuning, or employing more sophisticated models could be explored. Nonetheless, the current model provides a solid foundation for predicting passenger survival on the Titanic dataset.

### Recommendations

1. **Feature Engineering**: Explore additional features or interactions between existing features to provide the model with more relevant information.
2. **Hyperparameter Tuning**: Although Naive Bayes has few hyperparameters, exploring different configurations or variations (e.g., Bernoulli Naive Bayes) may yield better results.
3. **Ensemble Methods**: Combining the predictions of multiple models (e.g., using a voting classifier) could improve overall performance.
4. **Cross-Validation**: Implementing cross-validation during model training can provide more reliable performance estimates and help prevent overfitting.
